In [ ]:
# default_exp core.camera

# Camera
> A camera file with a range of functions to handle the camera and it's coordinates.

In [ ]:
import numpy as np
import torch
from runningpose.core.utils import wrap
from runningpose.quaternion import qrot, qinverse